# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 13 2023 10:29AM,257229,10,Sen-9153338,"Senseonics, Incorporated",Released
1,Feb 13 2023 10:27AM,257228,15,30013954,"Alliance Pharma, Inc.",Released
2,Feb 13 2023 10:27AM,257228,15,30013955,"Alliance Pharma, Inc.",Released
3,Feb 13 2023 10:27AM,257228,15,30013956,"Alliance Pharma, Inc.",Released
4,Feb 13 2023 10:27AM,257228,15,30013957,"Alliance Pharma, Inc.",Released
5,Feb 13 2023 10:27AM,257219,10,0086372652,ISDIN Corporation,Released
6,Feb 13 2023 10:27AM,257219,10,0086375544,ISDIN Corporation,Released
7,Feb 13 2023 10:27AM,257219,10,0086371279,ISDIN Corporation,Released
8,Feb 13 2023 10:27AM,257219,10,0086371310,ISDIN Corporation,Released
9,Feb 13 2023 10:27AM,257219,10,0086371311,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,257223,Released,8
28,257226,Released,10
29,257227,Released,6
30,257228,Released,4
31,257229,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257223,NaN,8.0
257226,NaN,10.0
257227,NaN,6.0
257228,NaN,4.0
257229,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257179,0.0,1.0
257186,0.0,3.0
257191,6.0,6.0
257194,40.0,1.0
257195,0.0,42.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257179,0,1
257186,0,3
257191,6,6
257194,40,1
257195,0,42


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257179,0,1
1,257186,0,3
2,257191,6,6
3,257194,40,1
4,257195,0,42


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257179,,1
1,257186,,3
2,257191,6,6
3,257194,40,1
4,257195,,42


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated"
1,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc."
5,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation
67,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation
117,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc."
123,Feb 13 2023 10:24AM,257226,10,"Methapharm, Inc."
133,Feb 13 2023 10:10AM,257222,15,"Alliance Pharma, Inc."
226,Feb 13 2023 10:10AM,257223,10,Emerginnova
234,Feb 13 2023 10:04AM,257221,21,"NBTY Global, Inc."
235,Feb 13 2023 9:57AM,257220,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated",,1
1,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc.",,4
2,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation,,62
3,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation,,50
4,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc.",,6
5,Feb 13 2023 10:24AM,257226,10,"Methapharm, Inc.",,10
6,Feb 13 2023 10:10AM,257222,15,"Alliance Pharma, Inc.",2,91
7,Feb 13 2023 10:10AM,257223,10,Emerginnova,,8
8,Feb 13 2023 10:04AM,257221,21,"NBTY Global, Inc.",,1
9,Feb 13 2023 9:57AM,257220,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated",1,
1,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc.",4,
2,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation,62,
3,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation,50,
4,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc.",6,
5,Feb 13 2023 10:24AM,257226,10,"Methapharm, Inc.",10,
6,Feb 13 2023 10:10AM,257222,15,"Alliance Pharma, Inc.",91,2
7,Feb 13 2023 10:10AM,257223,10,Emerginnova,8,
8,Feb 13 2023 10:04AM,257221,21,"NBTY Global, Inc.",1,
9,Feb 13 2023 9:57AM,257220,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated",1,
1,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc.",4,
2,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation,62,
3,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation,50,
4,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc.",6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated",1,NaN
1,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc.",4,NaN
2,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation,62,NaN
3,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation,50,NaN
4,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc.",6,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated",1,0.0
1,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc.",4,0.0
2,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation,62,0.0
3,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation,50,0.0
4,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc.",6,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4629747,386,0.0
12,514423,3,0.0
15,771677,101,2.0
19,1028778,50,46.0
21,514441,2,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4629747,386,0.0
1,12,514423,3,0.0
2,15,771677,101,2.0
3,19,1028778,50,46.0
4,21,514441,2,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,386,0.0
1,12,3,0.0
2,15,101,2.0
3,19,50,46.0
4,21,2,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,386.0
1,12,Released,3.0
2,15,Released,101.0
3,19,Released,50.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Executing,0.0,0.0,2.0,46.0,0.0
Released,386.0,3.0,101.0,50.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Executing,0.0,0.0,2.0,46.0,0.0
1,Released,386.0,3.0,101.0,50.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Executing,0.0,0.0,2.0,46.0,0.0
1,Released,386.0,3.0,101.0,50.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()